# Sentinel multiband stack
    Author: Ben Ross
    Approx Date: 24 June 2020

Description: Access previously downloaded sat data from the raster-collection script and enable manipulation of that data by correctly addressing all the contents.
The general idea is to unzip the sentinel rasters and read the contents. Converting the singleband files into a multiband image for the scripts further down the line. This is a means of reducing file handling complexity for future scripts.

In [13]:
import glob
import zipfile
import fnmatch
import rasterio

In [14]:
contentsfolder = r'E:\SATELLITE_DATA'
zipped_downloads = glob.glob(contentsfolder+'\*.zip', recursive=True)
zipped_downloads

for x in zipped_downloads:
    print('Unzipping {x}'.format(x=x))
    with zipfile.ZipFile(x,"r") as zip_ref:
        zip_ref.extractall(contentsfolder+"\Extracted")

print('Unzipping complete')

Unzipping E:\SATELLITE_DATA\S2A_MSIL2A_20211011T000251_N0301_R030_T56JNR_20211011T015715.zip
Unzipping E:\SATELLITE_DATA\S2A_MSIL2A_20211027T235251_N0301_R130_T56JNR_20211028T014413.zip
Unzipping complete


In [16]:
extracted_downloads = glob.glob(contentsfolder+'\Extracted\*.SAFE', recursive=True)
extracted_downloads

for x in extracted_downloads:
    #generate a list called y in which all the JPEG2000 files within one sat tile are listed
#     y = glob.glob(x+'\GRANULE\*\IMG_DATA\**\*.jp2', recursive=True)
    y = glob.glob(x+'\GRANULE\*\IMG_DATA\R20m\*.jp2', recursive=True)
    #print('There are {y} JPEG2000 files in the target folder {x}. These will be stacked using rasterio into a single {y} band image.'.format(y=len(y), x=x))
    
    #reorganise the list so that the B8A is between 8 and 9. Also omit the TCI.
    file_list = [y[0], y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[12], y[8], y[9], y[10], y[11]]

    #x = x[:-5]
    #start stacking the images using rasterio
    with rasterio.open(file_list[0]) as src0:
        meta = src0.meta

        # Update meta to reflect the number of layers
        meta.update(count = len(file_list))

        # Read each layer and write it to stack
        with rasterio.open(x + '_stack.tif', 'w', **meta) as dst:
            for id, layer in enumerate(file_list, start=1):
                with rasterio.open(layer) as src1:
                    dst.write_band(id, src1.read(1))
                    
print('Stacking complete')

ValueError: the array's dtype 'uint8' does not match the file's dtype 'uint16'

In [10]:
with rasterio.open(file_list[0]) as src:
    print(src.meta)

{'driver': 'JP2OpenJPEG', 'dtype': 'uint16', 'nodata': None, 'width': 5490, 'height': 5490, 'count': 1, 'crs': CRS.from_epsg(32756), 'transform': Affine(20.0, 0.0, 499980.0,
       0.0, -20.0, 7100020.0)}


In [5]:
meta

{'driver': 'JP2OpenJPEG',
 'dtype': 'uint16',
 'nodata': None,
 'width': 5490,
 'height': 5490,
 'count': 13,
 'crs': CRS.from_epsg(32756),
 'transform': Affine(20.0, 0.0, 499980.0,
        0.0, -20.0, 7100020.0)}